# OBJECTIVE
#### To create an apache cassandra database based on the queries given by the analytics team of the startup 'Sparkify' in order to fetch song play data to answer the questions. For that, I first created an ETL pipeline to preprocess the data. After that, with the help of the queries, I created three tables in order to retrieve information to answer the questions.

# Preprocessing of the file using ETL pipeline.

#### Import Python packages 

In [17]:
# Importing Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

# Retrieving the path to our event_data.
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collecting each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)
    print(len(file_path_list))

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files in file_path_list to create a csv file with fewer columns and non-empty rows that will be use for apache cassandra tables.

In [3]:
# Initializing an empty list.
full_data_rows_list = [] 
     
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:  
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # Storing data line by line in full_data_rows_list.      
        for line in csvreader:
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))

print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        # Writing data in event_datafile_new.csv for selected features.  
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# checking the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Creating tables as well as inserting and selecting data based on our queries.

#### Creating a Cluster

In [18]:
# This should make a connection to a Cassandra instance our local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating Keyspace

In [19]:
#Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project
    WITH REPLICATION =
    {'class': 'SimpleStrategy', 'replication_factor': 1}"""
)
except Exception as e:
    print(e)

#### Setting Keyspace

In [20]:
#Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project')
except Exception as e:
    print(e)

### Creating queries to ask the following three questions given by the analytical team of Sparkify.

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# i) Query-1

In [5]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
table1= "CREATE TABLE IF NOT EXISTS song_info_by_session"

# Taking session_id and itemInSession as composite primary keys in order to store 
# each row in node/s with unique identity.
query = table1+'(artist text, song text, length float, sessionId int,\
                 itemInSession int,\
                 PRIMARY KEY(sessionId,itemInSession))'
try:
    session.execute(query)
    
except Exception as e:
    
    print(e)        

                    

In [6]:
file = 'event_datafile_new.csv'

#Reading the event_datafile_new.csv
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO song_info_by_session (artist, song, length, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        ## Assigning values for each column in table song_info_by_session. 
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

In [7]:
## Selecting data based on the condition given in query-1.
query = "SELECT artist,song,length FROM song_info_by_session \
        WHERE sessionid=338 \
        AND iteminsession=4"

try:
    rows = session.execute(query)

except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# ii) Query-2

In [8]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
table2= "CREATE TABLE IF NOT EXISTS user_song_info_by_userid"

# Taking user_id and session_id as composite partition keys in order to break the data into chunks so that retrieval \
# process can be facilitated and hotspotting can be avoided.
# Taking item_in_session as clustering column in order to sort the data within the partition keys and also \
# to retrive the rows efficiently.
query = table2+'(artist text, song text, first_name text, last_name text,\
                 user_id int, session_id int, \
                 item_in_session int,\
                 PRIMARY KEY((user_id,session_id),item_in_session))'
try:
    session.execute(query)
    
except Exception as e:
    
    print(e)        

                    

In [9]:
file = 'event_datafile_new.csv'

with open(file,encoding='utf8') as f:
    csv_file = csv.reader(f)
    next(csv_file)
    for l in csv_file:
        ## Assigning the INSERT statements into the `query` variable
        query_2 = 'INSERT INTO user_song_info_by_userid (artist, song, first_name, last_name, user_id, session_id, item_in_session)'
        ## Assigning values for each column in table song_info_by_session. 
        query_2 = query_2 + 'VALUES (%s, %s, %s, %s, %s, %s, %s)'
        session.execute(query_2, (l[0], l[9], l[1], l[4], int(l[10]), int(l[8]), int(l[3])))
                    

In [10]:
## Selecting data based on the condition given in query-2.
query = "SELECT artist,song,first_name,last_name, item_in_session FROM user_song_info_by_userid \
         WHERE user_id=10 AND session_id=182"

try:
    rows = session.execute(query)

except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist,row.song,row.first_name,row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


# iii) Query-3

In [27]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
table3= "CREATE TABLE IF NOT EXISTS user_info_by_song"

# Taking user_id and song feature as composite partition key in order to divide data in small chunks and facilitate
# the retrieval process and also to avoid hotspotting.
query = table3+'(first_name text, last_name text,\
                 user_id int, song text, \
                 PRIMARY KEY((song,user_id)))'

# Creating an index on column song in table_3 in order to avoid ALLOW FILTERING.
index_query = 'CREATE INDEX ON table_3 (song);'

try:
    session.execute(query)
    
except Exception as e:
    
    print(e)
    
try:
    session.execute(index_query)
    
except Exception as e:
    
    print(e)

Error from server: code=2200 [Invalid query] message="Index table_3_song_idx_1 is a duplicate of existing index table_3_song_idx"


In [28]:
file = 'event_datafile_new.csv'

with open(file,encoding='utf8') as f:
    csv_file = csv.reader(f)
    next(csv_file)
    for l in csv_file:
        ## Assigning the INSERT statements into the `query` variable
        query_3 = 'INSERT INTO user_info_by_song (first_name, last_name, song, user_id)'
        ## Assigning values for each column in table song_info_by_session. 
        query_3 = query_3 + 'VALUES (%s, %s, %s, %s)'
        session.execute(query_3, (l[1], l[4], l[9], int(l[10])))

In [29]:
## Selecting data based on the condition given in query-3.
query = "SELECT first_name,last_name FROM user_info_by_song \
         WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(query)

except Exception as e:
    print(e)
    
for row in rows:
    print(row.first_name,row.last_name)

Sara Johnson
Tegan Levine
Jacqueline Lynch


### Queries to drop tables.

In [25]:
## Dropping the table before closing out the sessions
drop_table_1 = 'DROP TABLE IF EXISTS song_info_by_session'
try:
    rows = session.execute(drop_table_1)
except Exception as e:
    print(e)
    
drop_table_2 = 'DROP TABLE IF EXISTS user_song_info_by_userid'
try:
    rows = session.execute(drop_table_2)
except Exception as e:
    print(e)

drop_table_3 = 'DROP TABLE IF EXISTS user_info_by_song'
try:
    session.execute(drop_table_3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()